# Import Libraries


In [1]:
import tkinter as tk
from tkinter import messagebox
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector

# NLP and Evaluation Functions

In [2]:
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

def keyword_match(answer, reference):
    answer_tokens = set(preprocess(answer).split())
    reference_tokens = set(preprocess(reference).split())
    if not reference_tokens:
        return 0
    return len(answer_tokens & reference_tokens) / len(reference_tokens)

def semantic_similarity(answer, reference):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([answer, reference])
    return cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

def evaluate_answer(answer, reference, alpha=0.5):
    keyword_score = keyword_match(answer, reference)
    semantic_score = semantic_similarity(answer, reference)
    final_score = alpha * semantic_score + (1 - alpha) * keyword_score
    return {
        "Keyword Match Score": round(keyword_score, 2),
        "Semantic Similarity Score": round(semantic_score, 2),
        "Final Score": round(final_score, 2)
    }


# Database connection

In [3]:
def connect_database():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="nlp"
        )
        if conn.is_connected():
            print("Connected to MySQL database")
        return conn
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Java Answer Evaluation App

In [4]:
class AnswerEvaluationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Answer Evaluation")
        self.root.geometry("500x500")
        self.root.resizable(False, False)
        
        # Styling
        self.bg_color = "#f2f2f2"
        self.button_color = "#4CAF50"
        self.button_hover_color = "#45a049"
        self.font = ("Helvetica", 12)

        self.current_index = 0
        self.questions = self.fetch_all_questions()
        self.evaluation_results = {}  # Store results per question
        self.student_answers = {}     # Store student answers per question

        self.main_frame = tk.Frame(self.root, bg=self.bg_color) #mainframe 
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

        self.question_label = tk.Label(self.main_frame, text="Question:", font=("Helvetica", 14, "bold"), bg=self.bg_color)
        self.question_label.grid(row=0, column=0, sticky="w", pady=(10, 5))

        self.question_text = tk.Label(self.main_frame, text="", wraplength=400, bg=self.bg_color, font=self.font) 
        self.question_text.grid(row=1, column=0, sticky="w", pady=5)

        self.student_answer_label = tk.Label(self.main_frame, text="Student's Answer:", font=("Helvetica", 14, "bold"), bg=self.bg_color)
        self.student_answer_label.grid(row=2, column=0, sticky="w", pady=(10, 5))

        self.student_answer_entry = tk.Entry(self.main_frame, width=50, font=self.font)
        self.student_answer_entry.grid(row=3, column=0, pady=5)

        nav_frame = tk.Frame(self.main_frame, bg=self.bg_color)
        nav_frame.grid(row=4, column=0, pady=20)

        self.prev_button = tk.Button(nav_frame, text="Prev", command=self.load_prev_question, font=self.font, bg=self.button_color, fg="white", width=10, relief="raised", activebackground=self.button_hover_color)
        self.prev_button.grid(row=0, column=0, padx=10)

        self.next_button = tk.Button(nav_frame, text="Next", command=self.load_next_question, font=self.font, bg=self.button_color, fg="white", width=10, relief="raised", activebackground=self.button_hover_color)
        self.next_button.grid(row=0, column=1, padx=10)

        self.finish_button = tk.Button(self.main_frame, text="Finish", command=self.show_final_results, font=self.font, bg=self.button_color, fg="white", width=20, relief="raised", activebackground=self.button_hover_color, state="disabled")
        self.finish_button.grid(row=5, column=0, pady=(10, 5))

        self.load_question_by_index()

    def fetch_all_questions(self):
        conn = connect_database()
        if conn:
            cursor = conn.cursor()
            try:
                query = "SELECT question, answer FROM `java_questions`;"
                cursor.execute(query)
                results = cursor.fetchall()
                return results
            finally:
                cursor.close()
                conn.close()
        return []

    def load_question_by_index(self):
        if 0 <= self.current_index < len(self.questions):
            self.question_text_value, self.reference_answer = self.questions[self.current_index]
            self.question_text.config(text=self.question_text_value)

            # Load existing answer if available
            previous_answer = self.student_answers.get(self.current_index, "")
            self.student_answer_entry.delete(0, tk.END)
            self.student_answer_entry.insert(0, previous_answer)

            self.prev_button.config(state="normal" if self.current_index > 0 else "disabled")
            self.next_button.config(state="normal" if self.current_index < len(self.questions) - 1 else "disabled")
            self.finish_button.config(state="normal" if self.current_index == len(self.questions) - 1 else "disabled")
        else:
            self.question_text.config(text="No more questions.")
            self.student_answer_entry.config(state="disabled")
            self.prev_button.config(state="disabled")
            self.next_button.config(state="disabled")
            self.finish_button.config(state="normal")

    def evaluate_current_answer(self):
        student_answer = self.student_answer_entry.get()
        self.student_answers[self.current_index] = student_answer  # Save input

        if self.reference_answer:
            result = evaluate_answer(student_answer, self.reference_answer)
            self.evaluation_results[self.current_index] = {
                "Question": self.question_text_value,
                "Student Answer": student_answer,
                "Reference Answer": self.reference_answer,
                "Keyword Score": result['Keyword Match Score'],
                "Semantic Score": result['Semantic Similarity Score'],
                "Final Score": result['Final Score']
            }
        else:
            messagebox.showerror("Error", "Reference answer not available.")

    def load_next_question(self):
        if self.current_index < len(self.questions):
            self.evaluate_current_answer()
            self.current_index += 1
            self.load_question_by_index()

    def load_prev_question(self):
        self.evaluate_current_answer()
        if self.current_index > 0:
            self.current_index -= 1
            self.load_question_by_index()

    def show_final_results(self):
        self.evaluate_current_answer()  # Ensure latest answer is captured

        result_window = tk.Toplevel(self.root)
        result_window.title("Final Evaluation Summary")
        result_window.geometry("600x400")
        result_window.resizable(False, False)

        tk.Label(result_window, text="Final Evaluation Summary", font=("Helvetica", 16, "bold")).pack(pady=10)

        text_area = tk.Text(result_window, wrap="word", width=80, height=20, font=self.font)
        text_area.pack(padx=10, pady=10)

        # Add a scrollbar to the text area
        scroll_bar = tk.Scrollbar(result_window, command=text_area.yview)
        scroll_bar.pack(side=tk.RIGHT, fill=tk.Y)
        text_area.config(yscrollcommand=scroll_bar.set)

        for idx in sorted(self.evaluation_results.keys()):
            res = self.evaluation_results[idx]
            text_area.insert(tk.END, f"Q{idx + 1}: {res['Question']}\n")
            text_area.insert(tk.END, f"Student Answer: {res['Student Answer']}\n")
            text_area.insert(tk.END, f"Reference Answer: {res['Reference Answer']}\n")
            text_area.insert(tk.END, f"Keyword Score: {res['Keyword Score']}, Semantic Score: {res['Semantic Score']}, Final Score: {res['Final Score']}\n\n")

        text_area.config(state="disabled")

# Python Answer Evaluation App

In [5]:
class PythonAnswerEvaluationApp(AnswerEvaluationApp):
    def fetch_all_questions(self):  # Overriding to fetch Python questions
        conn = connect_database()
        if conn:
            cursor = conn.cursor()
            try:
                query = "SELECT question, answer FROM `python_questions`;"
                cursor.execute(query)
                results = cursor.fetchall()
                return results
            finally:
                cursor.close()
                conn.close()
        return []

# Main Menu Interface

In [ ]:
def open_java_interface():
    java_window = tk.Toplevel()
    AnswerEvaluationApp(java_window)

def open_python_interface():
    python_window = tk.Toplevel()
    PythonAnswerEvaluationApp(python_window)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Language Selector")
    root.geometry("300x300")
    root.resizable(False, False)

    java_button = tk.Button(root, text="Java", width=20, height=2, command=open_java_interface,
                            font=("Helvetica", 14), bg="#f5f6db", fg="black", relief="raised", activebackground="#f8fba2")
    java_button.pack(pady=(50, 50))

    python_button = tk.Button(root, text="Python", width=20, height=2, command=open_python_interface, 
                              font=("Helvetica", 14), bg="#f5f6db", fg="black", relief="raised", activebackground="#f8fba2")
    python_button.pack(pady=(50, 15))

    root.mainloop()
